# Geo Coding (緯度経度取得)・最寄駅情報取得を行う

In [1]:
import pandas as pd
import numpy as np
import urllib
import xml.etree.ElementTree as ET
import re
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import time
import pickle

In [2]:
# データベースの情報
server = 'gciteam16.database.windows.net'
database = 'mynavi-database'
username = 'gciteam16'
password = 'Password0'
port =1433

In [3]:
# 接続エンジンの作成
engine_config = "?driver=ODBC+Driver+13+for+SQL+Server?charset=shift-jis"
db_settings = {
    "host": server,
    "database": database,
    "user": username,
    "password": password,
    "port":port,
    "config_query":engine_config
}
engine = create_engine('mssql+pyodbc://{user}:{password}@{host}:{port}/{database}{config_query}'.format(**db_settings))

In [4]:
"""
# データの読み込み
sql_query = "SELECT * FROM main_table"
data = pd.read_sql_query(sql_query, con = engine)
"""

'\n# \xe3\x83\x87\xe3\x83\xbc\xe3\x82\xbf\xe3\x81\xae\xe8\xaa\xad\xe3\x81\xbf\xe8\xbe\xbc\xe3\x81\xbf\nsql_query = "SELECT * FROM main_table"\ndata = pd.read_sql_query(sql_query, con = engine)\n'

In [4]:
# コネクターの作成
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [5]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0]) 
    row = cursor.fetchone()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Aug 29 2017 13:06:11 
	Copyright (C) 2017 Microsoft Corporation



In [2]:
# https://drive.google.com/open?id=1lN_XnVuwtxGdZRGPFxnZTVgeG6ehmC8K に上がってるファイルは一発で読める
data = pd.read_csv("../../gci_project_storage/mynavi_data.csv", encoding="SHIFT-JIS", sep=",")
data.shape

(46469, 110)

<h3>目次：</h3><br>
・raw_data_table の作成<br>
・物件の Master Table の作成<br>
・最寄駅情報の抽出<br>
・駅の Master Table の作成<br>
・物件の Geo Coding<br>
・駅の情報を main_table に追加<br>


<h3>raw_data_table の作成

In [32]:
table_name = "raw_data_table"
#data.to_sql(table_name, con=engine, if_exists="fail", index=False)

<h3>物件の Master Table の作成

In [3]:
addresses = pd.Series(data.address.unique())
df_new = pd.DataFrame(index=addresses.index, columns=["longitude","latitude"])
df_new = pd.concat([addresses, df_new], axis=1)
df_new.columns = ["address","longitude","latitude"]
df_new.shape
data_address = df_new

In [83]:
table_name = "address_master"
df_new.to_sql(table_name, con=engine, if_exists="fail", index=False)

<h3>物件の Geo Coding

In [7]:
sql_query = "SELECT * FROM address_master"
data = pd.read_sql_query(sql_query, con = engine)

In [5]:
data_address.dtypes

address      object
longitude    object
latitude     object
dtype: object

In [14]:
address = data_address.address[0].encode("utf-8")
print(address)
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'
url = serviceurl + urllib.parse.urlencode({'sensor':'false', 'address': address})
print(url)

b'\xe6\x9d\xb1\xe4\xba\xac\xe9\x83\xbd\xe6\x96\x87\xe4\xba\xac\xe5\x8c\xba\xe6\x9c\xac\xe9\x83\xb7\xef\xbc\x94\xe4\xb8\x81\xe7\x9b\xae'
http://maps.googleapis.com/maps/api/geocode/xml?sensor=false&address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%96%87%E4%BA%AC%E5%8C%BA%E6%9C%AC%E9%83%B7%EF%BC%94%E4%B8%81%E7%9B%AE


<br><br><br>
# 0~1000
<br><br><br>

In [4]:
result_list_0_1000 = []

In [5]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(0,1000):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_0_1000.append(data)
    time.sleep(3)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%96%87%E4%BA%AC%E5%8C%BA%E6%9C%AC%E9%83%B7%EF%BC%94%E4%B8%81%E7%9B%AE&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%96%87%E4%BA%AC%E5%8C%BA%E6%9C%AC%E9%A7%92%E8%BE%BC%EF%BC%94%E4%B8%81%E7%9B%AE&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%96%87%E4%BA%AC%E5%8C%BA%E9%9F%B3%E7%BE%BD%EF%BC%91%E4%B8%81%E7%9B%AE&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%96%87%E4%BA%AC%E5%8C%BA%E5%8D%83%E7%9F%B3%EF%BC%94%E4%B8%81%E7%9B%AE&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%96%87%E4%BA%AC%E5%8C%BA%E6%9C%AC%E9%83%B7%EF%BC%94%E4%B8%81%E7%9B%AE2609&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E

In [6]:
len(result_list_0_1000)

1000

In [7]:
with open('../../gci_project_storage/result_list_0_1000.pickle', mode='wb') as f:
#    pickle.dump(result_list_0_1000, f)

In [21]:
with open('../../gci_project_storage/result_list_0_1000.pickle', mode='rb') as f:
    result_list_0_1000 = pickle.load(f)

In [22]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_0_1000[0:1000]]) == None)

0

<br><br><br>
# 1000~2000
<br><br><br>

In [6]:
result_list_1000_2000 = []

In [7]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(1000,2000):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_1000_2000.append(data)
    time.sleep(3)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E5%8F%B0%E6%9D%B1%E5%8C%BA%E4%B8%8A%E9%87%8E%EF%BC%95%E4%B8%81%E7%9B%AE%EF%BC%940%EF%BC%91%EF%BC%95&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E5%8F%B0%E6%9D%B1%E5%8C%BA%E6%B5%85%E8%8D%89%E6%A9%8B%EF%BC%95%E4%B8%81%E7%9B%AE12016&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E5%8F%B0%E6%9D%B1%E5%8C%BA%E4%B8%8A%E9%87%8E%EF%BC%91%E4%B8%81%E7%9B%AE&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E5%8F%B0%E6%9D%B1%E5%8C%BA%E4%B8%8A%E9%87%8E%EF%BC%91%E4%B8%81%E7%9B%AE305&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E5%8F%B0%E6%9D%B1%E5%8C%BA%E6%9D%B1%E4%B8%8A%E9%87%8E%EF%BC%96%E4%B8%81%E7%9B%AE2708&sensor=false
Retrieving http://maps.googleapis.com/maps

In [8]:
len(result_list_1000_2000)

1000

In [9]:
with open('../../gci_project_storage/result_list_1000_2000.pickle', mode='wb') as f:
#    pickle.dump(result_list_1000_2000, f)

In [19]:
with open('../../gci_project_storage/result_list_1000_2000.pickle', mode='rb') as f:
    result_list_1000_2000 = pickle.load(f)

In [20]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_1000_2000[0:1000]]) == None)

0

# 2000~4000

In [4]:
result_list_2000_4000 = []

In [5]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(2000,4000):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_2000_4000.append(data)
    time.sleep(5)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E7%9B%AE%E9%BB%92%E5%8C%BA%E7%9B%AE%E9%BB%92%EF%BC%91%E4%B8%81%E7%9B%AE3025&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E7%9B%AE%E9%BB%92%E5%8C%BA%E4%BA%94%E6%9C%AC%E6%9C%A8%EF%BC%93%E4%B8%81%E7%9B%AE1012&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E7%9B%AE%E9%BB%92%E5%8C%BA%E4%B8%AD%E7%9B%AE%E9%BB%92%EF%BC%92%E4%B8%81%E7%9B%AE805&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E7%9B%AE%E9%BB%92%E5%8C%BA%E5%8E%9F%E7%94%BA%EF%BC%91%E4%B8%81%E7%9B%AE16010&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E7%9B%AE%E9%BB%92%E5%8C%BA%E4%B8%89%E7%94%B0%EF%BC%92%E4%B8%81%E7%9B%AE18010&sensor=false
Retrieving http://maps.googleapis.com/maps/api/geocode/xml?ad

In [6]:
len(result_list_2000_4000)

2000

In [7]:
with open('../../gci_project_storage/result_list_2000_4000.pickle', mode='wb') as f:
    pickle.dump(result_list_2000_4000, f)

In [15]:
with open('../../gci_project_storage/result_list_2000_4000.pickle', mode='rb') as f:
    result_list_2000_4000 = pickle.load(f)

In [17]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_2000_4000[0:2000]]) == None)

0

# 4000~6400

In [5]:
result_list_4000_6400 = []

In [6]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(4000,6400):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    #print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_4000_6400.append(data)
    time.sleep(6)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

In [7]:
len(result_list_4000_6400)

2400

In [8]:
with open('../../gci_project_storage/result_list_4000_6400.pickle', mode='wb') as f:
#    pickle.dump(result_list_4000_6400, f)

In [9]:
with open('../../gci_project_storage/result_list_4000_6400.pickle', mode='rb') as f:
    result_list_4000_6400 = pickle.load(f)

In [14]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_4000_6400[0:2400]]) == None)

0

In [46]:
#文字化け対策
import sys
reload(sys)
print(sys.getdefaultencoding())
sys.setdefaultencoding('utf-8')

#query = "select top 5 * from address_master;" 
query = "UPDATE address_master SET longitude = {}, latitude = {} WHERE address = \"{}\";".format(lng,lat,address)

cursor.execute(query)
row = cursor.fetchone() 
while row: 
    print(row[0]) 
    row = cursor.fetchone()

ProgrammingError: ('42S22', "[42S22] [Microsoft][ODBC Driver 13 for SQL Server][SQL Server]Invalid column name '\xe6\x9d\xb1\xe4\xba\xac\xe9\x83\xbd\xe6\x96\x87\xe4\xba\xac\xe5\x8c\xba\xe6\x9c\xac\xe9\x83\xb7\xef\xbc\x94\xe4\xb8\x81\xe7\x9b\xae'. (207) (SQLExecDirectW)")

In [38]:
address = data.address[0].encode("utf-8")
print(address)
print(data.address[0].encode("utf-8"))
print("あいうえお")

In [41]:
query.encode("cp437")


UnicodeEncodeError: 'charmap' codec can't encode characters in position 90-100: character maps to <undefined>

In [23]:
sys.stdout.encoding

'cp437'

<h3>最寄駅情報の抽出

In [105]:
# 駅名と徒歩何分かを取得する関数
remove_bracket = lambda text :re.sub(u'\(.+?\)','',re.sub(u'（.+?）', '',re.sub(u'「」', '',text))) 
remove_question_mark = lambda text :re.sub(u'\?','',text)
remove_noise = lambda text:remove_bracket(remove_question_mark(text))
remove_dot = lambda text :re.sub(u'・.+?線','',text)#"・中央線東京駅"みたいなのを除く

get_station_info_0 = lambda text: (
    re.findall(u"線(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text)) 
    + re.findall(u"ライン(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text)) 
    + re.findall(u"ライナー(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text))
)
get_station_info = lambda text:{remove_dot(t[0]):int(t[1]) for t in get_station_info_0(text)} if len(get_station_info_0(text)) > 0 else {}
get_station_name = lambda text:[remove_dot(t[0]) for t in get_station_info_0(text)] if len(get_station_info_0(text)) > 0 else []
station_names = []

# 駅の総数を調べる
count_no_st = 0
for text in data.traffic:
    station_names += get_station_name(text)
    if(len(get_station_name(text))==0):
        count_no_st += 1
station_names = pd.Series(pd.Series(station_names).unique())

print("# Station : {}".format(len(station_names)))
print("# Rental Without Station : {}".format(count_no_st)) #＜-徒歩でなく、バスで移動の物件はNoneが返り値

# Station : 312
# Rental Without Station : 81


In [106]:
print(remove_bracket(data["traffic"][1758]))
print(get_station_name(data["traffic"][1758])[0])

中央線?御茶ノ水駅?徒歩0分
御茶ノ水駅


In [108]:
data.traffic[data.traffic.apply(lambda text: u"イナー" in get_station_name(text) )]

Series([], Name: traffic, dtype: object)

In [109]:
text = data.traffic[245]
print(text)
for st_name in get_station_name(text):
    print(st_name)
# u"[線(ライン)(ライナー)](\S+?駅).*?徒歩([0-9]+?)分"をfindall
print(remove_noise(text))
print(re.findall(u"ライナー(.+?駅).*?徒歩([0-9]+?)分",remove_noise(text))[0][0])

山手線日暮里駅?徒歩10分千代田線西日暮里駅?徒歩7分日暮里・舎人ライナー西日暮里(舎人ライナー)駅?徒歩7分
日暮里駅
西日暮里駅
西日暮里駅
山手線日暮里駅徒歩10分千代田線西日暮里駅徒歩7分日暮里・舎人ライナー西日暮里駅徒歩7分
西日暮里駅


In [79]:
station_info = data.traffic.apply(get_station_info)

In [80]:
station_info.head()

0    {u'春日駅': 2, u'後楽園駅': 4, u'本郷三丁目駅': 10}
1    {u'田端駅': 10, u'千駄木駅': 12, u'本駒込駅': 13}
2                 {u'春日駅': 7, u'本郷三丁目駅': 8}
3                 {u'新御徒町駅': 9, u'田原町駅': 7}
4    {u'田端駅': 10, u'千駄木駅': 12, u'本駒込駅': 13}
Name: traffic, dtype: object

<h3>駅の Master Table の作成

In [110]:
station_master = pd.DataFrame({ 
    'station_name' : station_names,
    'altitude' : None,
    'latitude' : None
})

In [113]:
table_name = "station_master"
station_master.to_sql(table_name, con=engine, if_exists="replace", index=False)

<h3>駅の情報を main_table に追加

In [199]:
# 一番近くの駅までの所要時間を求める関数（実は使わない）
get_min_time_to_st = lambda arg_dict: min(arg_dict.values()) if len(arg_dict)>0 else None
station_info.apply(get_min_time_to_st).head()

0     2.0
1    10.0
2     7.0
3     7.0
4    10.0
Name: traffic, dtype: float64

In [202]:
# 近くの駅までの所要時間の逆数のデータフレーム（説明変数に使う）
df_new = pd.DataFrame(index=data.index, columns=station_names).astype(np.float)

# 駅ごとにそれらを求める関数(徒歩0分の場合は、inverse:=2とした)
get_inverse_dist_to_st = lambda arg_dict, st_name : (1.0/arg_dict[st_name] if arg_dict[st_name]!=0 else 2) if st_name in arg_dict.keys() else 0

In [203]:
for st_name in station_names:
    this_st = st_name
    get_inverse_dist_to_this_st = lambda arg_dict: get_inverse_dist_to_st(arg_dict ,this_st)
    df_new[st_name] = station_info.apply(get_inverse_dist_to_this_st)

In [214]:
list(data.columns)

[u'traffic',
 u'address',
 u'rent',
 u'area',
 u'deposit',
 u'key_money',
 u'brokerage_fee',
 u'date',
 u'layout',
 u'layout_detail',
 u'depreciation',
 u'facility',
 u'flooring',
 u'guarantee',
 u'guarantor',
 u'insurance',
 u'renewal_fee',
 u'administration_fee',
 u'remark',
 u'locality',
 u'buildings_height',
 u'buildings_undergrand',
 u'surroundings',
 u'other_expence',
 u'parking',
 u'parking_detail',
 u'reform',
 u'status',
 u'structure',
 u'transaction_type',
 u'total_units',
 u'balcony',
 u'air_conditioner',
 u'auto_lock',
 u'bath_toilet',
 u'reheating',
 u'wash_basin',
 u'washing_machine',
 u'top_floor',
 u'corner_room',
 u'immediate',
 u'upper',
 u'shower',
 u'bath_drier',
 u'washlet',
 u'toilet_style',
 u'bathhouse',
 u'water_heater',
 u'system_kitchen',
 u'counter_kitchen',
 u'independent_kitchen',
 u'L_kitchen',
 u'refrigerator',
 u'gas_stove',
 u'IH_stove',
 u'electric_stove',
 u'stove_num',
 u'office',
 u'instrument',
 u'togather',
 u'share',
 u'foreigner',
 u'pet',
 u'e

In [63]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

print("inputting \"\" terminates this program")

while True:
    address = input('Enter location: ')
    if len(address) < 1 : break
    
    # Python3 系：urllib.parse.urlencode()
    url = serviceurl + urllib.urlencode({'sensor':'false', 'address': address})
    print ('Retrieving', url)
    uh = urllib.request.urlopen(url)
    data = uh.read()
    print ('Retrieved',len(data),'characters')
    print (data)
    print ()
    tree = ET.fromstring(data)


    results = tree.findall('result')
    lat = results[0].find('geometry').find('location').find('lat').text
    lng = results[0].find('geometry').find('location').find('lng').text
    location = results[0].find('formatted_address').text

    print ('lat',lat,'lng',lng)
    print (location)
    

inputting "" terminates this program
Enter location: tokyo


NameError: name 'tokyo' is not defined